In [ ]:

from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gym
import gym.spaces
import csv
import datetime


eps = 1e-8

def index_to_date(index):

    return (start_datetime + datetime.timedelta(index)).strftime(date_format)


def date_to_index(date_string):

    return (datetime.datetime.strptime(date_string, date_format) - start_datetime).days

def normalize(x):
    """ Create a universal normalization function across close/open ratio

    Args:
        x: input of any shape

    Returns: normalized data

    """
    return (x - 1) * 100


def sharpe(returns, freq=30, rfr=0):
    """ Given a set of returns, calculates naive (rfr=0) sharpe (eq 28). """
    return (np.sqrt(freq) * np.mean(returns - rfr + eps)) / np.std(returns - rfr + eps)


def max_drawdown(returns):
    """ Max drawdown. See https://www.investopedia.com/terms/m/maximum-drawdown-mdd.asp """
    peak = returns.max()
    trough = returns[returns.argmax():].min()
    return (trough - peak) / (peak + eps)

def p99_drawdown(returns):

    peak = np.percentile(returns, 99, interpolation='nearest')
    percidx = list(returns).index(peak)
    trough = returns[percidx:].min()
    return (trough - peak) / (peak + eps)

def p95_drawdown(returns):

    peak = np.percentile(returns, 95, interpolation='nearest')
    percidx = list(returns).index(peak)
    trough = returns[percidx:].min()
    return (trough - peak) / (peak + eps)

def p90_drawdown(returns):

    peak = np.percentile(returns, 90, interpolation='nearest')
    percidx = list(returns).index(peak)
    trough = returns[percidx:].min()
    return (trough - peak) / (peak + eps)

def under_water(returns):
    df = pd.DataFrame(returns, columns=('A',))
    df['cummax'] = df['A'].cummax()
    df['underwater'] = df['A'] < df['cummax']
    total_days = df['underwater'].sum()
    c = 0
    list_c =[]
    for i in df['underwater'].values:
        if i==False:        
            c = 0
        if i== True:
            c+=1
            list_c.append(c)
    max_days = max(list_c)
    return total_days, max_days

class DataGenerator(object):
    """Acts as data provider for each new episode."""
    

    def __init__(self, history, abbreviation, steps=730, window_length=50, start_idx=0, start_date=None):
        """

        Args:
            history: (num_stocks, timestamp, 5) open, high, low, close, volume
            abbreviation: a list of length num_stocks with assets name
            steps: the total number of steps to simulate, default is 2 years
            window_length: observation window, must be less than 50
            start_date: the date to start. Default is None and random pick one.
                        It should be a string e.g. '2012-08-13'
        """
        assert history.shape[0] == len(abbreviation), 'Number of stock is not consistent'
        import copy
        self.step = 0
        self.steps = steps + 1
        self.window_length = window_length
        self.start_idx = start_idx
        self.start_date = start_date
        self._data = history.copy() 
        self.asset_names = copy.copy(abbreviation)
        self.data = self._data
        self.idx = np.random.randint(low = self.window_length, high = self._data.shape[1] - self.steps)
        self.transition_matrix = 0
        self.discrete_win_size = 0
        self.mark_min = 0
        self.fin_un = 0
    def _step(self):

        self.step += 1
        obs = self.data[:, self.step:self.step + self.window_length, :].copy()

        ground_truth_obs = self.data[:, self.step + self.window_length:self.step + self.window_length + 1, :].copy()

        done = self.step >= self.steps
        return obs, done, ground_truth_obs

    
    def reset(self):
        self.step = 0

       
        if self.start_date is None:
            self.idx = np.random.randint(
                low = self.window_length, high = self._data.shape[1] - self.steps)
        else:
           
            self.idx = date_to_index(self.start_date) - self.start_idx
            assert self.idx >= self.window_length and self.idx <= self._data.shape[1] - self.steps, \
                'Invalid start date, must be window_length day after start date and simulation steps day before end date'

        data = self._data[:, self.idx - self.window_length:self.idx + self.steps + 1, :4]
        ########################################################
        #transition_matrix
        markov_states_raw = self._data[:, :, 3:4] / self._data[:, :, 0:1]
        markov_states = []
        for i in range(3, markov_states_raw.shape[1]):
            stepData = []
            c = np.array([markov_states_raw[:,k,0]  for k in range(i - 3, i)])
            stepData = normalize(c.T)
            markov_states.append(stepData)
            
        markov_states = np.array(markov_states)
        
        self.mark_min = [np.min(markov_states[:,0,:]),np.min(markov_states[:,1,:]),np.min(markov_states[:,2,:])]
        discrete_size = 6
        discrete_win_size_ass1 = (np.max(markov_states[:,0,:]) - np.min(markov_states[:,0,:])) /discrete_size
        discrete_win_size_ass2 = (np.max(markov_states[:,1,:]) - np.min(markov_states[:,1,:])) /discrete_size
        discrete_win_size_ass3 = (np.max(markov_states[:,2,:]) - np.min(markov_states[:,2,:])) /discrete_size
        self.discrete_win_size = [discrete_win_size_ass1,discrete_win_size_ass2,discrete_win_size_ass3]

        finals = []
        for fe in markov_states:
            ass1 = fe[0]
            ass2 = fe[1]
            ass3 = fe[2]
            ass1f = []
            ass2f = []
            ass3f = []
            final = []
            for i in range(3):
                bin1 = np.floor((ass1[i]- np.min(markov_states[:,0,:]))/discrete_win_size_ass1 )
                bin1 = int(bin1)
                ass1f.append(bin1)
          

                bin2 = np.floor((ass2[i]- np.min(markov_states[:,1,:]))/discrete_win_size_ass2 )
                bin2 = int(bin2)
                ass2f.append(bin2)
      

                bin3 = np.floor((ass3[i]- np.min(markov_states[:,2,:]))/discrete_win_size_ass3 )
                bin3 = int(bin3)
                ass3f.append(bin3)

                fgds = [ass1f,ass2f,ass3f]
                final.append(fgds)
                
            finals.append(final[-1])
        finals = np.array(finals)
        
        self.fin_un = np.unique(finals,axis =0)

        
        mark1 = finals.reshape(finals.shape[0],9)
        mark3ext = ["{}{}{}{}{}{}{}{}{}".format(i, j,k,l,m,o,ij,ku,gw) for i, j,k,l,m,o,ij,ku,gw in mark1]
        df = pd.DataFrame(columns = ['state', 'next_state'])
        for q, val in enumerate(mark3ext [:-1]): # We don't care about last state
            df_stg = pd.DataFrame(index=[0])
            df_stg['state'], df_stg['next_state'] = mark3ext[q], mark3ext[q+1]
            df = pd.concat([df, df_stg], axis = 0)
        cross_tab = pd.crosstab(df['state'], df['next_state'])
        tabtabtab = cross_tab.div(cross_tab.sum(axis=1), axis=0)
        transition_matrix = tabtabtab.unstack().reorder_levels(('state','next_state'))
        self.transition_matrix = transition_matrix
         
        # apply augmentation?
        self.data = data
        
        return self.data[:, self.step:self.step + self.window_length, :].copy(), \
               self.data[:, self.step + self.window_length:self.step + self.window_length + 1, :].copy()
        
        

class PortfolioSim(object):
    """
    Portfolio management sim.
    Params:
    - cost e.g. 0.0025 is max in Poliniex
    Based of [Jiang 2017](https://arxiv.org/abs/1706.10059)
    """

    def __init__(self, asset_names=list(), steps=730, trading_cost=0.0025, time_cost=0.0):
        self.asset_names = asset_names
        self.cost = trading_cost
        self.time_cost = time_cost
        self.steps = steps
        self.p0 = 0
        self.infos = []

    def _step(self, w1, y1):
        """
        Step.
        w1 - new action of portfolio weights - e.g. [0.1,0.9,0.0]
        y1 - price relative vector also called return
            e.g. [1.0, 0.9, 1.1]
        Numbered equations are from https://arxiv.org/abs/1706.10059
        """
        assert w1.shape == y1.shape, 'w1 and y1 must have the same shape'
        assert y1[0] == 1.0, 'y1[0] must be 1'

        #p0 = self.p0

        dw1 = (y1 * w1) / (np.dot(y1, w1) + eps)  # (eq7) weights evolve into

        mu1 = self.cost * (np.abs(dw1 - w1)).sum()  # (eq16) cost to change portfolio

        assert mu1 < 1.0, 'Cost is larger than current holding'

        p1 = self.p0 * (1 - mu1) * np.dot(y1, w1)  # (eq11) final portfolio value

        p1 = p1 * (1 - self.time_cost)  # we can add a cost to holding

        rho1 = p1 / self.p0 - 1  # rate of returns
        r1 = np.log((p1 + eps) / (self.p0 + eps))  # log rate of return
        reward = r1 / self.steps * 1000.  # (22) average logarithmic accumulated return
        # remember for next step
        self.p0 = p1

        # if we run out of money, we're done (losing all the money)
        done = p1 == 0

        info = {
            "reward": reward,
            "log_return": r1,
            "portfolio_value": p1,
            "return": y1.mean(),
            "rate_of_return": rho1,
            "weights_mean": w1.mean(),
            "weights_std": w1.std(),
            "cost": mu1,
        }
        self.infos.append(info)
        return reward, info, done

    def reset(self):
        self.infos = []
        self.p0 = 1.0


class PortfolioEnv(gym.Env):
    """
    An environment for financial portfolio management.
    Financial portfolio management is the process of constant redistribution of a fund into different
    financial products.
    Based on [Jiang 2017](https://arxiv.org/abs/1706.10059)
    """

    metadata = {'render.modes': ['human', 'ansi']}

    def __init__(self,
                 history,
                 abbreviation,
                 steps=730,  # 2 years
                 trading_cost=0.0025,
                 time_cost=0.00,
                 window_length=50,
                 start_idx=0,
                 sample_start_date=None
                 ):
        """
        An environment for financial portfolio management.
        Params:
            steps - steps in episode
            scale - scale data and each episode (except return)
            augment - fraction to randomly shift data by
            trading_cost - cost of trade as a fraction
            time_cost - cost of holding as a fraction
            window_length - how many past observations to return
            start_idx - The number of days from '2012-08-13' of the dataset
            sample_start_date - The start date sampling from the history
        """
        self.window_length = window_length
        self.num_stocks = history.shape[0]
        self.start_idx = start_idx
        self.df_portfolio_performance = None        

        self.src = DataGenerator(history, abbreviation, steps=steps, window_length=window_length, start_idx=start_idx,
                                 start_date=sample_start_date)

        self.sim = PortfolioSim(
            asset_names=abbreviation,
            trading_cost=trading_cost,
            time_cost=time_cost,
            steps=steps)

        # openai gym attributes
        # action will be the portfolio weights from 0 to 1 for each asset
        self.action_space = gym.spaces.Box(
            0, 1, shape=(len(self.src.asset_names) + 1,), dtype=np.float32)  # include cash

        # get the observation space from the data min and max
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(len(abbreviation), window_length,
                                                                                 history.shape[-1]), dtype=np.float32)
        self.infos = []
        
    def step(self, action):
        return self._step(action)

    def _step(self, action):

        np.testing.assert_almost_equal(
            action.shape,
            (len(self.sim.asset_names) + 1,)
        )

 
        action = np.clip(action, 0, 1)

        weights = action  
        weights /= (weights.sum() + eps)
        weights[0] += np.clip(1 - weights.sum(), 0, 1) 


        observation, done1, ground_truth_obs = self.src._step()


        cash_observation = np.ones((1, self.window_length, observation.shape[2]))
        observation = np.concatenate((cash_observation, observation), axis=0)

        cash_ground_truth = np.ones((1, 1, ground_truth_obs.shape[2]))
        ground_truth_obs = np.concatenate((cash_ground_truth, ground_truth_obs), axis=0)


        close_price_vector = observation[:, -1, 3]
        open_price_vector = observation[:, -1, 0]
        y1 = close_price_vector / open_price_vector
        reward, info, done2 = self.sim._step(weights, y1)

 
        info['market_value'] = np.cumprod([inf["return"] for inf in self.infos + [info]])[-1]
  
        info['date'] = index_to_date(self.start_idx + self.src.idx + self.src.step)
        info['steps'] = self.src.step
        info['next_obs'] = ground_truth_obs

        self.infos.append(info)

        return observation, reward, done1 or done2, info
    
    def reset(self):
        return self._reset()

    def _reset(self):
        self.infos = []
        self.sim.reset()
        observation, ground_truth_obs = self.src.reset()
        cash_observation = np.ones((1, self.window_length, observation.shape[2]))
        observation = np.concatenate((cash_observation, observation), axis=0)
        cash_ground_truth = np.ones((1, 1, ground_truth_obs.shape[2]))
        ground_truth_obs = np.concatenate((cash_ground_truth, ground_truth_obs), axis=0)
        info = {}
        info['next_obs'] = ground_truth_obs
        return observation, info

    def _render(self, mode='human', close=False):
        if close:
            return
        if mode == 'ansi':
            pprint(self.infos[-1])
        elif mode == 'human':
            return self.plot()
            
    def render(self, mode='human', close=False):
        return self._render(mode='human', close=False)

    def plot(self):

        df_info = pd.DataFrame(self.infos)
        df_info['date'] = pd.to_datetime(df_info['date'], format='%Y-%m-%d')
        df_info.set_index('date', inplace=True)
        mdd = max_drawdown(df_info.rate_of_return + 1)
        sharpe_ratio = sharpe(df_info.rate_of_return)
        title = 'max_drawdown={: 2.2%} sharpe_ratio={: 2.4f}'.format(mdd, sharpe_ratio)       
        df_info[["portfolio_value", "market_value"]].plot(title=title, fig=plt.gcf(), rot=30)
        


class MultiActionPortfolioEnv(PortfolioEnv):
    def __init__(self,
                 history,
                 abbreviation,
                 model_names,
                 steps=730,  # 2 years
                 trading_cost=0.0025,
                 time_cost=0.00,
                 window_length=50,
                 start_idx=0,
                 sample_start_date=None,
                 ):
        super(MultiActionPortfolioEnv, self).__init__(history, abbreviation, steps, trading_cost, time_cost, window_length,
                              start_idx, sample_start_date)
        self.model_names = model_names
        self.sim = [PortfolioSim(
            asset_names=abbreviation,
            trading_cost=trading_cost,
            time_cost=time_cost,
            steps=steps) for _ in range(len(self.model_names))]
        self.infos = []
    def _step(self, action):
        """ Step the environment by a vector of actions

        Args:
            action: (num_models, num_stocks + 1)

        Returns:

        """
        assert action.ndim == 2, 'Action must be a two dimensional array with shape (num_models, num_stocks + 1)'
        assert action.shape[1] == len(self.sim[0].asset_names) + 1
        assert action.shape[0] == len(self.model_names)
 
        action = np.clip(action, 0, 1)
        weights = action 
        weights /= (np.sum(weights, axis=1, keepdims=True) + eps)

        weights[:, 0] += np.clip(1 - np.sum(weights, axis=1), 0, 1)
        assert ((action >= 0) * (action <= 1)).all(), 'all action values should be between 0 and 1. Not %s' % action
        np.testing.assert_almost_equal(np.sum(weights, axis=1), np.ones(shape=(weights.shape[0])), 3,
                                       err_msg='weights should sum to 1. action="%s"' % weights)
        observation, done1, ground_truth_obs = self.src._step()

        cash_observation = np.ones((1, self.window_length, observation.shape[2]))
        observation = np.concatenate((cash_observation, observation), axis=0)

        cash_ground_truth = np.ones((1, 1, ground_truth_obs.shape[2]))
        ground_truth_obs = np.concatenate((cash_ground_truth, ground_truth_obs), axis=0)

        close_price_vector = observation[:, -1, 3]
        open_price_vector = observation[:, -1, 0]
        y1 = close_price_vector / open_price_vector

        rewards = np.empty(shape=(weights.shape[0]))
        info = {}
        dones = np.empty(shape=(weights.shape[0]), dtype=bool)
        for i in range(weights.shape[0]):
            reward, current_info, done2 = self.sim[i]._step(weights[i], y1)
            rewards[i] = reward
            info[self.model_names[i]] = current_info['portfolio_value']
            info['return'] = current_info['return']
            dones[i] = done2

        info['market_value'] = np.cumprod([inf["return"] for inf in self.infos + [info]])[-1]
        info['date'] = index_to_date(self.start_idx + self.src.idx + self.src.step)
        info['steps'] = self.src.step
        info['next_obs'] = ground_truth_obs

        self.infos.append(info)

        return observation, rewards, np.all(dones) or done1, info

    def _reset(self):
        self.infos = []
        for sim in self.sim:
            sim.reset()
        observation, ground_truth_obs = self.src.reset()
        cash_observation = np.ones((1, self.window_length, observation.shape[2]))
        observation = np.concatenate((cash_observation, observation), axis=0)
        cash_ground_truth = np.ones((1, 1, ground_truth_obs.shape[2]))
        ground_truth_obs = np.concatenate((cash_ground_truth, ground_truth_obs), axis=0)
        info = {}
        info['next_obs'] = ground_truth_obs
        return observation, info

    def plot(self):
        df_info = pd.DataFrame(self.infos)
        fig=plt.gcf()
        title = 'Trading Performance of Models'
        df_info['date'] = pd.to_datetime(df_info['date'], format='%Y-%m-%d')
        df_info.set_index('date', inplace=True)
        
        mdd = max_drawdown(df_info[self.model_names].values + 1)
        p99dd = p99_drawdown(df_info[self.model_names].values + 1)
        p95dd = p95_drawdown(df_info[self.model_names].values + 1)
        p90dd = p90_drawdown(df_info[self.model_names].values + 1)
        ttduw, mduw = under_water(df_info[self.model_names].values)
        sharpe_ratio = sharpe(df_info[self.model_names].values)
        per_dic = {"MDD" : mdd , "99DD" : p99dd , "95DD": p95dd, "90DD":p90dd,"TTDUW" : ttduw,"MDUW":mduw, "SHRPR": sharpe_ratio}
        
        bmdd = max_drawdown(df_info[['market_value']].values + 1)
        bp99dd = p99_drawdown(df_info[['market_value']].values + 1)
        bp95dd = p95_drawdown(df_info[['market_value']].values + 1)
        bp90dd = p90_drawdown(df_info[['market_value']].values + 1)
        bttduw, bmduw = under_water(df_info[['market_value']].values)
        bsharpe_ratio = sharpe(df_info[['market_value']].values)
        b_dic = {"MDD" : bmdd , "99DD" : bp99dd , "95DD": bp95dd, "90DD":bp90dd,"TTDUW" : bttduw,"MDUW":bmduw, "SHRPR": bsharpe_ratio}

        title = 'Max drawdown={: 2.2%}  99th percentile drawdown={: 2.2%}  95th percentile drawdown={: 2.2%}  90th percentile drawdown={: 2.2%}   Sharpe Ratio={: 2.4f}'.format(mdd,p99dd,p95dd,p90dd,sharpe_ratio)       
        df_info[self.model_names + ['market_value']].plot(title=title, fig=fig, rot=30)
        return per_dic, b_dic, df_info[self.model_names + ['market_value']]


In [ ]:
import tensorflow as tf

class ActorNetwork(object):


    def __init__(self, sess, state_dim, action_dim, action_bound, learning_rate, tau, batch_size):

        self.sess = sess
        self.s_dim = state_dim
        self.a_dim = action_dim
        self.action_bound = action_bound
        self.learning_rate = learning_rate
        self.tau = tau
        self.batch_size = batch_size

  
        self.inputs, self.out, self.scaled_out = self.create_actor_network()

        self.network_params = tf.trainable_variables()

  
        self.target_inputs, self.target_out, self.target_scaled_out = self.create_actor_network()

        self.target_network_params = tf.trainable_variables()[
                                     len(self.network_params):]

 
        self.update_target_network_params = \
            [self.target_network_params[i].assign(tf.multiply(self.network_params[i], self.tau) +
                                                  tf.multiply(self.target_network_params[i], 1. - self.tau))
             for i in range(len(self.target_network_params))]


        self.action_gradient = tf.placeholder(tf.float32, [None] + self.a_dim)


        self.unnormalized_actor_gradients = tf.gradients(
            self.scaled_out, self.network_params, -self.action_gradient)
        self.actor_gradients = list(map(lambda x: tf.div(x, self.batch_size), self.unnormalized_actor_gradients))


        self.optimize = tf.train.AdamOptimizer(self.learning_rate). \
            apply_gradients(zip(self.actor_gradients, self.network_params))

        self.num_trainable_vars = len(self.network_params) + len(self.target_network_params)

    def create_actor_network(self):
        raise NotImplementedError('Create actor should return (inputs, out, scaled_out)')

    def train(self, inputs, a_gradient):
        self.sess.run(self.optimize, feed_dict={
            self.inputs: inputs,
            self.action_gradient: a_gradient
        })

    def predict(self, inputs):
        return self.sess.run(self.scaled_out, feed_dict={
            self.inputs: inputs
        })

    def predict_target(self, inputs):
        return self.sess.run(self.target_scaled_out, feed_dict={
            self.target_inputs: inputs
        })

    def update_target_network(self):
        self.sess.run(self.update_target_network_params)

    def get_num_trainable_vars(self):
        return self.num_trainable_vars


In [ ]:

import tensorflow as tf
import tflearn


class CriticNetwork(object):
    """
    Input to the network is the state and action, output is Q(s,a).
    The action must be obtained from the output of the Actor network.
    """

    def __init__(self, sess, state_dim, action_dim, learning_rate, tau, num_actor_vars):
        self.sess = sess
        assert isinstance(state_dim, list), 'state_dim must be a list.'
        self.s_dim = state_dim
        assert isinstance(action_dim, list), 'action_dim must be a list.'
        self.a_dim = action_dim
        self.learning_rate = learning_rate
        self.tau = tau


        self.inputs, self.action, self.out = self.create_critic_network()

        self.network_params = tf.trainable_variables()[num_actor_vars:]

        self.target_inputs, self.target_action, self.target_out = self.create_critic_network()

        self.target_network_params = tf.trainable_variables()[(len(self.network_params) + num_actor_vars):]

    
        self.update_target_network_params = \
            [self.target_network_params[i].assign(tf.multiply(self.network_params[i], self.tau) \
                                                  + tf.multiply(self.target_network_params[i], 1. - self.tau))
             for i in range(len(self.target_network_params))]


        self.predicted_q_value = tf.placeholder(tf.float32, [None, 1])

    
        self.loss = tflearn.mean_square(self.predicted_q_value, self.out)
        self.optimize = tf.train.AdamOptimizer(
            self.learning_rate).minimize(self.loss)

        self.action_grads = tf.gradients(self.out, self.action)

    def create_critic_network(self):
        raise NotImplementedError('Create critic should return (inputs, action, out)')

    def train(self, inputs, action, predicted_q_value):
        return self.sess.run([self.out, self.optimize], feed_dict={
            self.inputs: inputs,
            self.action: action,
            self.predicted_q_value: predicted_q_value
        })

    def predict(self, inputs, action):
        return self.sess.run(self.out, feed_dict={
            self.inputs: inputs,
            self.action: action
        })

    def predict_target(self, inputs, action):
        return self.sess.run(self.target_out, feed_dict={
            self.target_inputs: inputs,
            self.target_action: action
        })

    def action_gradients(self, inputs, actions):
        return self.sess.run(self.action_grads, feed_dict={
            self.inputs: inputs,
            self.action: actions
        })

    def update_target_network(self):
        self.sess.run(self.update_target_network_params)


In [ ]:
"""
Taken from https://github.com/openai/baselines/blob/master/baselines/ddpg/noise.py, which is
based on http://math.stackexchange.com/questions/1287634/implementing-ornstein-uhlenbeck-in-matlab
"""

import numpy as np

class OrnsteinUhlenbeckActionNoise:
    def __init__(self, mu, sigma=0.3, theta=.15, dt=1e-2, x0=None):
        self.theta = theta
        self.mu = mu
        self.sigma = sigma
        self.dt = dt
        self.x0 = x0
        self.reset()

    def __call__(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + \
            self.sigma * np.sqrt(self.dt) * np.random.normal(size=self.mu.shape)
        self.x_prev = x
        return x

    def reset(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros_like(self.mu)

    def __repr__(self):
        return 'OrnsteinUhlenbeckActionNoise(mu={}, sigma={})'.format(self.mu, self.sigma)


In [ ]:
from collections import deque
import random
import numpy as np


class ReplayBuffer(object):
    def __init__(self, buffer_size, random_seed=123):

        self.buffer_size = buffer_size
        self.count = 0
        self.buffer = deque()
        random.seed(random_seed)

    def add(self, s, a, r, t, s2):
        experience = (s, a, r, t, s2)
        if self.count < self.buffer_size:
            self.buffer.append(experience)
            self.count += 1
        else:
            self.buffer.popleft()
            self.buffer.append(experience)

    def size(self):
        return self.count

    def sample_batch(self, batch_size):
        if self.count < batch_size:
            batch = random.sample(self.buffer, self.count)
        else:
            batch = random.sample(self.buffer, batch_size)

        s_batch = np.array([_[0] for _ in batch])
        a_batch = np.array([_[1] for _ in batch])
        r_batch = np.array([_[2] for _ in batch])
        t_batch = np.array([_[3] for _ in batch])
        s2_batch = np.array([_[4] for _ in batch])

        return s_batch, a_batch, r_batch, t_batch, s2_batch

    def clear(self):
        self.buffer.clear()
        self.count = 0


In [ ]:
"""
The robust deep deterministic policy gradient model
"""

import cvxpy as cvx
import os
import traceback
import json
import numpy as np
import tensorflow as tf
import pandas as pd
from .replay_buffer import ReplayBuffer
from ..base_model import BaseModel


def build_summaries():
    episode_reward = tf.Variable(0.)
    tf.summary.scalar("Reward", episode_reward)
    episode_ave_max_q = tf.Variable(0.)
    tf.summary.scalar("Qmax_Value", episode_ave_max_q)

    summary_vars = [episode_reward, episode_ave_max_q]
    summary_ops = tf.summary.merge_all()

    return summary_ops, summary_vars

def cvx_optimizer(v,f,beta):
    ''' worst case optimization algorithm'''
    var = cvx.Variable(len(v))
    #objective = cvx.Minimize(var.T * v)
    objective = cvx.Maximize(var.T * v)
    problem = cvx.Problem(objective, [cvx.log(var).T * f >= (beta) , cvx.sum(var) == 1, var>= 0])
    problem.solve()            
    obj = objective.value

    return obj
    
class DDPG(BaseModel):
    def __init__(self, env, sess, actor, critic, actor_noise, obs_normalizer=None, action_processor=None,
                 config_file='config/default.json',
                 model_save_path='weights/ddpg/ddpg.ckpt', summary_path='results/ddpg/',beta=-10,lannda = 0.5):
        with open(config_file) as f:
            self.config = json.load(f)
        assert self.config != None, "Can't load config file"
        np.random.seed(self.config['seed'])
        if env:
            env.seed(self.config['seed'])
        self.model_save_path = model_save_path
        self.summary_path = summary_path
        self.sess = sess
        self.env = env
        self.actor = actor
        self.critic = critic
        self.actor_noise = actor_noise
        self.obs_normalizer = obs_normalizer
        self.action_processor = action_processor
        self.summary_ops, self.summary_vars = build_summaries()
        self.beta = beta
        self.lannda = lannda

    def initialize(self, load_weights=True, verbose=True):
        """ Load training history from path. To be add feature to just load weights, not training states

        """
        if load_weights:
            try:
                variables = tf.global_variables()
                param_dict = {}
                saver = tf.train.Saver()
                saver.restore(self.sess, self.model_save_path)
                for var in variables:
                    var_name = var.name[:-2]
                    if verbose:
                        print('Loading {} from checkpoint. Name: {}'.format(var.name, var_name))
                    param_dict[var_name] = var
            except:
                traceback.print_exc()
                print('Build model from scratch')
                self.sess.run(tf.global_variables_initializer())
        else:
            print('Build model from scratch')
            self.sess.run(tf.global_variables_initializer())

    def train(self, save_every_episode=1, verbose=True, debug=False):

        writer = tf.summary.FileWriter(self.summary_path, self.sess.graph)
        print('inside DDPG train')
        self.actor.update_target_network()
        self.critic.update_target_network()

        np.random.seed(self.config['seed'])
        num_episode = self.config['episode']
        batch_size = self.config['batch size']
        gamma = self.config['gamma']
        self.buffer = ReplayBuffer(self.config['buffer size'])

        # main training loop
        for i in range(num_episode):
            if verbose and debug:
                print("Episode: " + str(i) + " Replay Buffer " + str(self.buffer.count()))

            previous_observation = self.env.reset()
            transition_matrix = self.env.src.transition_matrix
            discrete_win_size = self.env.src.discrete_win_size
            mark_min = self.env.src.mark_min
            all_mark_states = self.env.src.fin_un
            if self.obs_normalizer:
                previous_observation = self.obs_normalizer(previous_observation)

            ep_reward = 0
            ep_ave_max_q = 0
            # keeps sampling until done
            for j in range(self.config['max step']):
                action = self.actor.predict(np.expand_dims(previous_observation, axis=0)).squeeze(
                    axis=0) + self.actor_noise()

                if self.action_processor:
                    action_take = self.action_processor(action)
                else:
                    action_take = action
                # step forward
                observation, reward, done, _ = self.env.step(action_take)

                if self.obs_normalizer:
                    observation = self.obs_normalizer(observation)

                # add to buffer
                self.buffer.add(previous_observation, action, reward, done, observation)

                if self.buffer.size() >= batch_size:
     
                    s_batch, a_batch, r_batch, t_batch, s2_batch = self.buffer.sample_batch(batch_size)
  
                    mark = s_batch[:,1:,:,0]
                    finals = []
                    for fe in mark:
                        ass1 = fe[0]
                        ass2 = fe[1]
                        ass3 = fe[2]
                        ass1f = []
                        ass2f = []
                        ass3f = []
                        final = []
                        for ik in range(3):
                            bin1 = np.floor((ass1[ik]- mark_min[0])/discrete_win_size[0] )
                            bin1 = int(bin1)
                            ass1f.append(bin1)
                            #ass1f.astype(int)

                            bin2 = np.floor((ass2[ik]- mark_min[1])/discrete_win_size[1] )
                            bin2 = int(bin2)
                            ass2f.append(bin2)
                            #ass2f.astype(int)

                            bin3 = np.floor((ass3[ik]- mark_min[2])/discrete_win_size[2] )
                            bin3 = int(bin3)
                            ass3f.append(bin3)

                            fgds = [ass1f,ass2f,ass3f]
                            final.append(fgds)
                          
                        finals.append(final[-1])
                    finals = np.array(finals)

                    mark1 = finals.reshape((finals.shape[0],9)).astype(int)
               
                    mark3ext = ["{}{}{}{}{}{}{}{}{}".format(u,o,p,m,n,w,ou,pm,nw) for u,o,p,m,n,w,ou,pm,nw in mark1]
        
                    all_mark_tokened = ["{}{}{}{}{}{}{}{}{}".format(h,d,y,g,x,z,hd,yg,xz) for h,d,y,g,x,z,hd,yg,xz in all_mark_states.reshape((all_mark_states.shape[0],9))]
                    
                    beta_max = []
                    transition_matrix_rows = []
                    for token in mark3ext:
                        tr_row = transition_matrix.loc[token].values
                        n_logartim = np.where(tr_row != 0, np.log(tr_row), 0)
           
                        beta_max.append(sum(tr_row * n_logartim))
                        transition_matrix_rows.append(tr_row)
                    
                    target_q = self.critic.predict_target(s2_batch, self.actor.predict_target(s2_batch))
                    
                    mark_plus = all_mark_states.reshape((all_mark_states.shape[0],all_mark_states.shape[1],all_mark_states.shape[2],1))
                    mark_plus = np.concatenate((np.zeros((all_mark_states.shape[0],1,all_mark_states.shape[2],1)),mark_plus),axis=1)
                    target_q_plus = self.critic.predict_target(mark_plus, self.actor.predict_target(mark_plus))
                    q_helper_plus = dict(zip(all_mark_tokened, target_q_plus))
                    q_helper = dict(zip(mark3ext,target_q))
                    for tokened in all_mark_tokened:
                        for tokener in mark3ext:
                            if tokened == tokener:
                                q_helper_plus[tokened] = q_helper[tokener]

                    v = np.array(q_helper_plus.values()).reshape(-1)
                    
                    
                    #markov_states_batch = list(map(int, raw_markov_states_batch))
                                                             
                    #convex_optimization
                    
                    #final_robust = cvx_batch_optimizer(v,transition_matrix_rows_batch)
                    #final_robust = np.array(final_robust).reshape(-1)
                    #print(final_robust)

                    y_i = []

                    landa = self.lannda/10 
                    for k in range(batch_size):
                        if t_batch[k]:
                            y_i.append(r_batch[k])
                        else:
                            try:
                                #print(len(transition_matrix_rows[k]))
                                final_robust = cvx_optimizer(v,transition_matrix_rows[k],self.beta)
                                y_i.append(r_batch[k] + landa * gamma * target_q[k] + gamma *(1-landa) * final_robust)
                            except:
                                y_i.append(r_batch[k] +  gamma * target_q[k])
                                    
                    
                    #print(y_i)
                    
                 
                    # Update the critic given the targets
                    predicted_q_value, _ = self.critic.train(
                        s_batch, a_batch, np.reshape(y_i, (batch_size, 1)))

                    ep_ave_max_q += np.amax(predicted_q_value)

                    # Update the actor policy using the sampled gradient
                    a_outs = self.actor.predict(s_batch)
                    grads = self.critic.action_gradients(s_batch, a_outs)
                    self.actor.train(s_batch, grads[0])

                    # Update target networks
                    self.actor.update_target_network()
                    self.critic.update_target_network()

                ep_reward += reward
                previous_observation = observation

                if done or j == self.config['max step'] - 1:
                    summary_str = self.sess.run(self.summary_ops, feed_dict={
                        self.summary_vars[0]: ep_reward,
                        self.summary_vars[1]: ep_ave_max_q / float(j)
                    })

                    writer.add_summary(summary_str, i)
                    writer.flush()

                    print('Episode: {:f}, Reward: {:.2f}, Qmax: {:.4f}'.format(i, ep_reward, (ep_ave_max_q / float(j))))
                    break
        print('save model.')
        self.save_model(verbose=True)
        print('Finish.')

    def predict(self, observation):
        """ predict the next action using actor model, only used in deploy.
            Can be used in multiple environments.

        Args:
            observation: (batch_size, num_stocks + 1, window_length)

        Returns: action array with shape (batch_size, num_stocks + 1)

        """
        if self.obs_normalizer:
            observation = self.obs_normalizer(observation)
        action = self.actor.predict(observation)
        if self.action_processor:
            action = self.action_processor(action)
        return action

    def predict_single(self, observation):
        """ Predict the action of a single observation

        Args:
            observation: (num_stocks + 1, window_length)

        Returns: a single action array with shape (num_stocks + 1,)

        """
        if self.obs_normalizer:
            observation = self.obs_normalizer(observation)
        action = self.actor.predict(np.expand_dims(observation, axis=0)).squeeze(axis=0)
        if self.action_processor:
            action = self.action_processor(action)
        return action

    def save_model(self, verbose=False):
        if not os.path.exists(self.model_save_path):
            os.makedirs(self.model_save_path, exist_ok=True)

        saver = tf.train.Saver()
        model_path = saver.save(self.sess, self.model_save_path)
        print("Model saved in %s" % model_path)


In [ ]:
"""
Use DDPG to train a stock trader based on a window of history price
"""



from model.ddpg.actor import ActorNetwork
from model.ddpg.critic import CriticNetwork
from model.ddpg.ddpg import DDPG
from model.ddpg.ornstein_uhlenbeck import OrnsteinUhlenbeckActionNoise

from environment.portfolio import PortfolioEnv
from utils.data import read_stock_history, normalize

import numpy as np
import tflearn
import tensorflow as tf
import argparse
import pprint

DEBUG = False


def get_model_path(window_length, predictor_type, use_batch_norm,idstocks):
    if use_batch_norm:
        batch_norm_str = 'batch_norm'
    else:
        batch_norm_str = 'no_batch_norm'
    return 'weights/stock/{}/window_{}/{}/{}/checkpoint.ckpt'.format(predictor_type, window_length, batch_norm_str,idstocks)


def get_result_path(window_length, predictor_type, use_batch_norm,idstocks):
    if use_batch_norm:
        batch_norm_str = 'batch_norm'
    else:
        batch_norm_str = 'no_batch_norm'
    return 'results/stock/{}/window_{}/{}/{}/'.format(predictor_type, window_length, batch_norm_str,idstocks)


def get_variable_scope(window_length, predictor_type, use_batch_norm):
    if use_batch_norm:
        batch_norm_str = 'batch_norm'
    else:
        batch_norm_str = 'no_batch_norm'
    return 'Robust_{}_w{}'.format(predictor_type, window_length)


def stock_predictor(inputs, predictor_type, use_batch_norm):
    window_length = inputs.get_shape()[2]
    assert predictor_type in ['cnn', 'lstm'], 'type must be either cnn or lstm'
    if predictor_type == 'cnn':
        net = tflearn.conv_2d(inputs, 32, (1, 3), padding='valid')
        if use_batch_norm:
            net = tflearn.layers.normalization.batch_normalization(net)
        net = tflearn.activations.relu(net)
        net = tflearn.conv_2d(net, 32, (1, window_length - 2), padding='valid')
        if use_batch_norm:
            net = tflearn.layers.normalization.batch_normalization(net)
        net = tflearn.activations.relu(net)
        if DEBUG:
            print('After conv2d:', net.shape)
        net = tflearn.flatten(net)
        if DEBUG:
            print('Output:', net.shape)
    elif predictor_type == 'lstm':
        num_stocks = inputs.get_shape()[1]
        hidden_dim = 32
        net = tflearn.reshape(inputs, new_shape=[-1, window_length, 1])
        if DEBUG:
            print('Reshaped input:', net.shape)
        net = tflearn.lstm(net, hidden_dim)
        if DEBUG:
            print('After LSTM:', net.shape)
        net = tflearn.reshape(net, new_shape=[-1, num_stocks, hidden_dim])
        if DEBUG:
            print('After reshape:', net.shape)
        net = tflearn.flatten(net)
        if DEBUG:
            print('Output:', net.shape)
    else:
        raise NotImplementedError

    return net


class StockActor(ActorNetwork):
    def __init__(self, sess, state_dim, action_dim, action_bound, learning_rate, tau, batch_size,
                 predictor_type, use_batch_norm):
        self.predictor_type = predictor_type
        self.use_batch_norm = use_batch_norm
        ActorNetwork.__init__(self, sess, state_dim, action_dim, action_bound, learning_rate, tau, batch_size)

    def create_actor_network(self):
        """
        self.s_dim: a list specifies shape
        """
        nb_classes, window_length = self.s_dim
        assert nb_classes == self.a_dim[0]
        assert window_length > 2, 'This architecture only support window length larger than 2.'
        inputs = tflearn.input_data(shape=[None] + self.s_dim + [1], name='input')

        net = stock_predictor(inputs, self.predictor_type, self.use_batch_norm)

        net = tflearn.fully_connected(net, 64)
        if self.use_batch_norm:
            net = tflearn.layers.normalization.batch_normalization(net)

        net = tflearn.activations.relu(net)
        net = tflearn.fully_connected(net, 64)
        if self.use_batch_norm:
            net = tflearn.layers.normalization.batch_normalization(net)

        net = tflearn.activations.relu(net)
 
        w_init = tflearn.initializations.uniform(minval=-0.003, maxval=0.003)
        out = tflearn.fully_connected(net, self.a_dim[0], activation='softmax', weights_init=w_init)
  
        scaled_out = tf.multiply(out, self.action_bound)
        return inputs, out, scaled_out

    def train(self, inputs, a_gradient):
        window_length = self.s_dim[1]
        inputs = inputs[:, :, -window_length:, :]
        self.sess.run(self.optimize, feed_dict={
            self.inputs: inputs,
            self.action_gradient: a_gradient
        })

    def predict(self, inputs):
        window_length = self.s_dim[1]
        inputs = inputs[:, :, -window_length:, :]
        return self.sess.run(self.scaled_out, feed_dict={
            self.inputs: inputs
        })

    def predict_target(self, inputs):
        window_length = self.s_dim[1]
        inputs = inputs[:, :, -window_length:, :]
        return self.sess.run(self.target_scaled_out, feed_dict={
            self.target_inputs: inputs
        })


class StockCritic(CriticNetwork):
    def __init__(self, sess, state_dim, action_dim, learning_rate, tau, num_actor_vars,
                 predictor_type, use_batch_norm):
        self.predictor_type = predictor_type
        self.use_batch_norm = use_batch_norm
        CriticNetwork.__init__(self, sess, state_dim, action_dim, learning_rate, tau, num_actor_vars)

    def create_critic_network(self):
        inputs = tflearn.input_data(shape=[None] + self.s_dim + [1])
        action = tflearn.input_data(shape=[None] + self.a_dim)

        net = stock_predictor(inputs, self.predictor_type, self.use_batch_norm)


        t1 = tflearn.fully_connected(net, 64)
        t2 = tflearn.fully_connected(action, 64)

        net = tf.add(t1, t2)
        if self.use_batch_norm:
            net = tflearn.layers.normalization.batch_normalization(net)
        net = tflearn.activations.relu(net)

        w_init = tflearn.initializations.uniform(minval=-0.003, maxval=0.003)
        out = tflearn.fully_connected(net, 1, weights_init=w_init)
        return inputs, action, out

    def train(self, inputs, action, predicted_q_value):
        window_length = self.s_dim[1]
        inputs = inputs[:, :, -window_length:, :]
        return self.sess.run([self.out, self.optimize], feed_dict={
            self.inputs: inputs,
            self.action: action,
            self.predicted_q_value: predicted_q_value
        })

    def predict(self, inputs, action):
        window_length = self.s_dim[1]
        inputs = inputs[:, :, -window_length:, :]
        return self.sess.run(self.out, feed_dict={
            self.inputs: inputs,
            self.action: action
        })

    def predict_target(self, inputs, action):
        window_length = self.s_dim[1]
        inputs = inputs[:, :, -window_length:, :]
        return self.sess.run(self.target_out, feed_dict={
            self.target_inputs: inputs,
            self.target_action: action
        })

    def action_gradients(self, inputs, actions):
        window_length = self.s_dim[1]
        inputs = inputs[:, :, -window_length:, :]
        return self.sess.run(self.action_grads, feed_dict={
            self.inputs: inputs,
            self.action: actions
        })


def obs_normalizer(observation):
    """ Preprocess observation obtained by environment

    Args:
        observation: (nb_classes, window_length, num_features) or with info

    Returns: normalized

    """
    if isinstance(observation, tuple):
        observation = observation[0]
    # directly use close/open ratio as feature
    observation = observation[:, :, 3:4] / observation[:, :, 0:1]
    observation = normalize(observation)

    
    return observation


def test_model(env, model):
    observation, info = env.reset()
    done = False
    while not done:
        action = model.predict_single(observation)
        observation, _, done, _ = env.step(action)
    env.render()


def test_model_multiple(env, models):
    observations_list = []
    actions_list = []
    info_list = []
    observation, info = env.reset()
    done = False
    while not done:
        actions = []
        for model in models:
            actions.append(model.predict_single(observation))
        actions = np.array(actions)
        observation, _, done, info = env.step(actions)
        observations_list.append(observation)
        actions_list.append(actions)
        info_list.append(info)
    model_dic, bench_dic, df_performance = env.render()
    return model_dic, bench_dic, observations_list, info_list, actions_list, df_performance


if __name__ == '__main__':

    DEBUG = False

    history = read_stock_history(filepath='/stock_data/history.csv')
    history = history[2:5,:,:4]
    tickers = read_stock_history(filepath='/stock_data/tickers.csv')
    tickers = tickers[2:5]
    idstocks = "".join(tickers)
    target_stocks = tickers
    num_training_time = 1095
    window_length = 3
    nb_classes = len(target_stocks) + 1

    target_history = np.empty(shape=(len(target_stocks), num_training_time, history.shape[2]))
    for i, stock in enumerate(target_stocks):
        target_history[i] = history[tickers.index(stock), :num_training_time, :]

    env = PortfolioEnv(target_history, target_stocks, steps=1000, window_length=window_length)

    action_dim = [nb_classes]
    state_dim = [nb_classes, window_length]
    batch_size = 64
    action_bound = 1.
    tau = 1e-3
    predictor_type = 'cnn'
    use_batch_norm = True

    actor_noise = OrnsteinUhlenbeckActionNoise(mu=np.zeros(action_dim))
    model_save_path = get_model_path(window_length, predictor_type, use_batch_norm,idstocks)
    summary_path = get_result_path(window_length, predictor_type, use_batch_norm,idstocks)

    variable_scope = get_variable_scope(window_length, predictor_type, use_batch_norm)

    with tf.variable_scope(variable_scope):
        sess = tf.Session()
        actor = StockActor(sess, state_dim, action_dim, action_bound, 1e-4, tau, batch_size,
                           predictor_type, use_batch_norm)
        critic = StockCritic(sess=sess, state_dim=state_dim, action_dim=action_dim, tau=1e-3,
                             learning_rate=1e-3, num_actor_vars=actor.get_num_trainable_vars(),
                             predictor_type=predictor_type, use_batch_norm=use_batch_norm)
        ddpg_model = DDPG(env, sess, actor, critic, actor_noise, obs_normalizer=obs_normalizer,
                          config_file='config/stock.json', model_save_path=model_save_path,
                          summary_path=summary_path)
        ddpg_model.initialize(load_weights=False)
        print('calling DDPG train')
        ddpg_model.train()
